<div align="center">
<h2 style="color:#a70000; font-weight:bold">How do we deploy these ML Model now?</h2>
<h3 style="color:#a70000; font-weight:bold">Python has a solution to it: Serialization!</h3>
</div>

In Python, pickling is a standard way to store objects and retrieve them as their original state. To give a simple example:

In [1]:
list_to_pickle = [1, 'here', 123, 'walker']

#Pickling the list
import dill as pickle

with open('./list.pk', 'wb') as filename:
    pickle.dump(list_to_pickle, filename)

In [2]:
!ls *.pk

list.pk


### Contents of the pickle file are nothing but all your data stored in bytes!

When we load the pickle back:

In [3]:
with open('./list.pk', 'rb') as filename:
    loaded_pickle = pickle.load(filename)
    
print("Content of the pickle: {}".format(loaded_pickle))

Content of the pickle: [1, 'here', 123, 'walker']


### For folks who are more familiar with R

$$.rda\;==\;.pk$$

<div align="center">
<h2 style="color:#a70000; font-weight:bold">For our hack session we already have a model, pickled and ready</h2>


<img src="http://download.gamezone.com/uploads/image/data/1220233/article_post_width_pick.JPG" height="500px" width="500px"/>
<div>

## How to load our pickled model

In [4]:
import dill as pickle
clf = 'model_v1.pk'

In [5]:
loaded_model = None
with open('../working_api/working_api/models/'+clf, 'rb') as f:
    loaded_model = pickle.load(f)

In [6]:
loaded_model

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('preprocessing', PreProcessing()), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impu..._jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__n_estimators': [10, 20, 30], 'randomforestclassifier__max_depth': [None, 6, 8, 10], 'randomforestclassifier__max_leaf_nodes': [None, 5, 10, 20], 'randomforestclassifier__min_impurity_split': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

### Test dataset:

In [7]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('../working_api/test_data/test1.csv', encoding="utf-8-sig")
df = df.head(10)

In [9]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
5,LP001054,Male,Yes,0,Not Graduate,Yes,2165,3422,152.0,360.0,1.0,Urban
6,LP001055,Female,No,1,Not Graduate,No,2226,0,59.0,360.0,1.0,Semiurban
7,LP001056,Male,Yes,2,Not Graduate,No,3881,0,147.0,360.0,0.0,Rural
8,LP001059,Male,Yes,2,Graduate,NaN,13633,0,280.0,240.0,1.0,Urban
9,LP001067,Male,No,0,Not Graduate,No,2400,2400,123.0,360.0,1.0,Semiurban


In [10]:
loaded_model.predict(df)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [11]:
loaded_model.predict_proba(df)

array([[ 0.23530068,  0.76469932],
       [ 0.1434071 ,  0.8565929 ],
       [ 0.11905397,  0.88094603],
       [ 0.12139402,  0.87860598],
       [ 0.29924693,  0.70075307],
       [ 0.28859075,  0.71140925],
       [ 0.16387166,  0.83612834],
       [ 0.77090254,  0.22909746],
       [ 0.24318158,  0.75681842],
       [ 0.12350885,  0.87649115]])

<div class="divider_h1" align="center">
<h1 style="font-weight:bold;"><font color="#5ea774">DEMO: HELLO-WORLD<font></h1>
</div>

- Importing the __Flask__ class

In [11]:
from flask import Flask

- Creating an instance of the class:

In [12]:
app = Flask(__name__)

- Since we are dealing with Web APIs and endpoints, __Flask__ saves us the hassle by providing decorators, `route()` as a means to add new endpoints to the web application:

In [13]:
@app.route('/')
def hello_world():
    return("Hello, world!")

- __What is a decorator?__

    * In layman's term, decorators are __Funceptions__. Functions within functions.
    * Example:

In [12]:
def decorate(func):
   def func_wrapper(name):
       return "Our result: {}".format(func(name))
   return func_wrapper

In [13]:
@decorate
def get_text(name):
   return "lorem ipsum, {} dolor sit amet".format(name)

In [14]:
get_text("John")

'Our result: lorem ipsum, John dolor sit amet'

- Content in `return()` in the above definition will be displayed in the browser.

<div class="divider_h1" align="center">
<h1 style="font-weight:bold;"><font color="#5ea774">DEMO:WRAPPING ML MODELS<font></h1>
</div>

### Client Side code:

In [15]:
import json
import requests

- #### Setting the headers to send and accept json responses

In [16]:
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

- #### Reading test batch

In [17]:
df = pd.read_csv('../working_api/test_data/test1.csv', encoding="utf-8-sig")
df = df.head(5)
print(df)

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001015   Male     Yes          0      Graduate            No   
1  LP001022   Male     Yes          1      Graduate            No   
2  LP001031   Male     Yes          2      Graduate            No   
3  LP001035   Male     Yes          2      Graduate            No   
4  LP001051   Male      No          0  Not Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5720                  0       110.0             360.0   
1             3076               1500       126.0             360.0   
2             5000               1800       208.0             360.0   
3             2340               2546       100.0             360.0   
4             3276                  0        78.0             360.0   

   Credit_History Property_Area  
0             1.0         Urban  
1             1.0         Urban  
2             1.0         Urban  
3             NaN     

- #### Converting Pandas Dataframe to json

In [18]:
data = df.to_json(orient='records')

In [19]:
data

'[{"Loan_ID":"LP001015","Gender":"Male","Married":"Yes","Dependents":"0","Education":"Graduate","Self_Employed":"No","ApplicantIncome":5720,"CoapplicantIncome":0,"LoanAmount":110.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":"Urban"},{"Loan_ID":"LP001022","Gender":"Male","Married":"Yes","Dependents":"1","Education":"Graduate","Self_Employed":"No","ApplicantIncome":3076,"CoapplicantIncome":1500,"LoanAmount":126.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":"Urban"},{"Loan_ID":"LP001031","Gender":"Male","Married":"Yes","Dependents":"2","Education":"Graduate","Self_Employed":"No","ApplicantIncome":5000,"CoapplicantIncome":1800,"LoanAmount":208.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":"Urban"},{"Loan_ID":"LP001035","Gender":"Male","Married":"Yes","Dependents":"2","Education":"Graduate","Self_Employed":"No","ApplicantIncome":2340,"CoapplicantIncome":2546,"LoanAmount":100.0,"Loan_Amount_Term":360.0,"Credit_History":null,"Property_Are

In [20]:
data = df.to_json(orient='split')

In [21]:
data

'{"columns":["Loan_ID","Gender","Married","Dependents","Education","Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History","Property_Area"],"index":[0,1,2,3,4],"data":[["LP001015","Male","Yes","0","Graduate","No",5720,0,110.0,360.0,1.0,"Urban"],["LP001022","Male","Yes","1","Graduate","No",3076,1500,126.0,360.0,1.0,"Urban"],["LP001031","Male","Yes","2","Graduate","No",5000,1800,208.0,360.0,1.0,"Urban"],["LP001035","Male","Yes","2","Graduate","No",2340,2546,100.0,360.0,null,"Urban"],["LP001051","Male","No","0","Not Graduate","No",3276,0,78.0,360.0,1.0,"Urban"]]}'

In [22]:
data = df.to_json(orient='columns')

In [23]:
data

'{"Loan_ID":{"0":"LP001015","1":"LP001022","2":"LP001031","3":"LP001035","4":"LP001051"},"Gender":{"0":"Male","1":"Male","2":"Male","3":"Male","4":"Male"},"Married":{"0":"Yes","1":"Yes","2":"Yes","3":"Yes","4":"No"},"Dependents":{"0":"0","1":"1","2":"2","3":"2","4":"0"},"Education":{"0":"Graduate","1":"Graduate","2":"Graduate","3":"Graduate","4":"Not Graduate"},"Self_Employed":{"0":"No","1":"No","2":"No","3":"No","4":"No"},"ApplicantIncome":{"0":5720,"1":3076,"2":5000,"3":2340,"4":3276},"CoapplicantIncome":{"0":0,"1":1500,"2":1800,"3":2546,"4":0},"LoanAmount":{"0":110.0,"1":126.0,"2":208.0,"3":100.0,"4":78.0},"Loan_Amount_Term":{"0":360.0,"1":360.0,"2":360.0,"3":360.0,"4":360.0},"Credit_History":{"0":1.0,"1":1.0,"2":1.0,"3":null,"4":1.0},"Property_Area":{"0":"Urban","1":"Urban","2":"Urban","3":"Urban","4":"Urban"}}'

- #### We'll be sending resources to the server-side, hence __POST__

```python
resp = requests.post("http://<url/ip>/predict", \
                    data = json.dumps(data),\
                    headers= header)
```

### Server Side code:

In [24]:
import pandas as pd

```python
test_json = response.get_json() 
#response object in Flask that contains our dataframe sent our via POST

test = pd.read_json(test_json, orient='records')
```

- #### Converting the json back to pandas dataframe

In [25]:
test = pd.read_json(data, orient='records')

In [26]:
test

,ApplicantIncome,CoapplicantIncome,Credit_History,Dependents,Education,Gender,LoanAmount,Loan_Amount_Term,Loan_ID,Married,Property_Area,Self_Employed
0,5720,0,1.0,0,Graduate,Male,110,360,LP001015,Yes,Urban,No
1,3076,1500,1.0,1,Graduate,Male,126,360,LP001022,Yes,Urban,No
2,5000,1800,1.0,2,Graduate,Male,208,360,LP001031,Yes,Urban,No
3,2340,2546,NaN,2,Graduate,Male,100,360,LP001035,Yes,Urban,No
4,3276,0,1.0,0,Not Graduate,Male,78,360,LP001051,No,Urban,No


- Minor pre-processing steps before we proceed:

In [27]:
#To resolve the issue of TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'
test['Dependents'] = [str(x) for x in list(test['Dependents'])]

#Getting the Loan_IDs separated out
loan_ids = test['Loan_ID']

- <p>Loading our pickled estimator and do predictions:</p>

In [28]:
import dill as pickle
clf = 'model_v1.pk'

In [29]:
loaded_model = None
with open('../working_api/working_api/models/'+clf, 'rb') as f:
    loaded_model = pickle.load(f)

In [30]:
predictions = loaded_model.predict(test)

In [31]:
predictions

array([1, 1, 1, 1, 1])

- <p><font color="#810000" style="font-weight:bold;">jsonify</font> our predictions and send the response back with status code: <font color="#810000" >200</font></p>

In [32]:
prediction_series = list(pd.Series(predictions))
final_predictions = pd.DataFrame(list(zip(loan_ids, prediction_series)))
print(final_predictions)

          0  1
0  LP001015  1
1  LP001022  1
2  LP001031  1
3  LP001035  1
4  LP001051  1


In [33]:
from flask import jsonify

```python
responses = jsonify(predictions=final_predictions.to_json(orient="records"))
responses.status_code = 200
```

<div align="center">
<h2 style="color:#a70000; font-weight:bold;">Demo Run for API: Local</h2>
</div>

In [50]:
import json
import requests
import pandas as pd

"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

"""Reading test batch
"""
df = pd.read_csv('../working_api/test_data/test1.csv', encoding="utf-8-sig")
df = df.head(10)

"""Converting Pandas Dataframe to json

MORE INFO:
==============
orient: string

    Series
    =====================
    default is 'index'
    allowed values are: {'split', 'records', 'index'}
    
    Dataframe
    =====================
    default is 'columns'
    allowed values are: {'split', 'records', 'columns', 'values'}
    
"""
data = df.to_json(orient='records')

"""POST <url>/predict, sending data to the Flask Server

NOTE: 
==================================
- <url> for local: localhost or 127.0.0.1 or 0.0.0.0
- <url> for cloud: DNS or ip of VM created
"""
resp = requests.post("http://104.236.105.217:5000/predict", \
                    data = json.dumps(data),\
                    headers= header)

In [51]:
resp.status_code

200

In [52]:
resp.json()

{'predictions': '[{"0":"LP001015","1":1},{"0":"LP001022","1":1},{"0":"LP001031","1":1},{"0":"LP001035","1":1},{"0":"LP001051","1":1},{"0":"LP001054","1":1},{"0":"LP001055","1":1},{"0":"LP001056","1":0},{"0":"LP001059","1":1},{"0":"LP001067","1":1}]'}

<div align="center">
<h2 style="color:#a70000; font-weight:bold;">Demo Run for API: Cloud</h2>
</div>

In [44]:
import json
import requests
import pandas as pd

"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

"""Reading test batch
"""
df = pd.read_csv('../working_api/test_data/test1.csv', encoding="utf-8-sig")
df = df.head(10)

"""Converting Pandas Dataframe to json

MORE INFO:
==============
orient: string

    Series
    =====================
    default is 'index'
    allowed values are: {'split', 'records', 'index'}
    
    Dataframe
    =====================
    default is 'columns'
    allowed values are: {'split', 'records', 'columns', 'values'}
    
"""
data = df.to_json(orient='records')

"""POST <url>/predict, sending data to the Flask Server

NOTE: 
==================================
- <url> for local: localhost or 127.0.0.1
- <url> for cloud: DNS or ip of VM created
"""
resp = requests.post("http://45.55.83.36:5000/predict", \
                    data = json.dumps(data),\
                    headers= header)

In [45]:
resp.status_code

200

In [46]:
resp.json()

{'predictions': '[{"0":"LP001015","1":1},{"0":"LP001022","1":1},{"0":"LP001031","1":1},{"0":"LP001035","1":1},{"0":"LP001051","1":1},{"0":"LP001054","1":1},{"0":"LP001055","1":1},{"0":"LP001056","1":0},{"0":"LP001059","1":1},{"0":"LP001067","1":1}]'}